### What even is an average?
For this project, a ounce of care should most likely be taken on what we mean when we say _RMSE_ as there are multiple ways one could define, particularly when taking zonal/meridional averages as well as averages over ensembles.


In [1]:
import os
import re 
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr
import cartopy.crs as ccrs



In [2]:
NAME = 'DJF'
FILE = 'jan'

BASE_DIR = f"/scratch/users/robcking/am4_error_growth_full/{FILE}"


In [3]:
full_memberlist = sorted(glob(os.path.join(BASE_DIR,'**','atmos_4xdaily.nc')),key=lambda x: int(re.search(r'\w+\/(\d+)\/\w+.nc',x).group(1)))
full_members =  xr.open_mfdataset(full_memberlist,combine='nested',concat_dim='members',chunks={'time':10,'pfull':'auto','grid_yt':'auto'}).isel(time=slice(250))

In [4]:
full_members

<xarray.Dataset>
Dimensions:       (time: 250, pfull: 33, phalf: 34, grid_yt: 360, members: 20,
                   bnds: 2, grid_xt: 576)
Coordinates:
  * time          (time) object 1979-07-01 06:00:00 ... 1979-09-01 12:00:00
  * pfull         (pfull) float64 2.164 5.845 10.75 17.11 ... 985.4 992.8 997.9
  * phalf         (phalf) float64 1.0 4.0 8.186 13.79 ... 989.7 995.9 1e+03
  * grid_yt       (grid_yt) float64 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * grid_xt       (grid_xt) float64 0.3125 0.9375 1.562 ... 358.4 359.1 359.7
Dimensions without coordinates: members, bnds
Data variables:
    grid_yt_bnds  (members, grid_yt, bnds) float64 dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    grid_xt_bnds  (members, grid_xt, bnds) float64 dask.array<chunksize=(1, 576, 2), meta=np.ndarray>
    ucomp         (members, time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 23, 252, 576), meta=np.ndarray>
    vcomp         (members, time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 23, 252, 576), meta=np.ndarray>
    u_ref         (members, time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 360, 576), meta=np.ndarray>
    v_ref         (members, time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 360, 576), meta=np.ndarray>
    t_ref         (members, time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 360, 576), meta=np.ndarray>
    temp          (members, time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 23, 252, 576), meta=np.ndarray>
    hght          (members, time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 10, 23, 252, 576), meta=np.ndarray>
    zsurf         (members, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
Attributes:
    title:                 c96L33_am4p0
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  2023.01.02
    git_hash:              8176c2b45079431f2053ec773253938f8aee4828
    creationtime:          Thu Mar  7 11:17:36 2024
    hostname:              sh03-05n25.int
    history:               fregrid --input_mosaic INPUT/C96_mosaic.nc --input...